# Kepler Framework, Emulate v8

In [1]:
from kepler.pandas.menu       import ElectronSequence as Chain
from kepler.pandas.readers    import load, load_in_loop, load_hdf
from core import Efficiency
from Gaugi import mkdir_p, progressbar
import numpy as np
import pandas as pd
import collections
import os
from pprint import pprint
from copy import deepcopy

import gc
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2

Welcome to JupyROOT 6.16/00
Using all sub packages with ROOT dependence
INFO: Pandarallel will run on 40 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.

Applying ATLAS style settings...

Applying ATLAS style settings...

Applying ATLAS style settings...


In [2]:
output_path = 'output/emulation'
mkdir_p(output_path)

In [3]:
dpath_data17 = '../data/output/data17_13TeV.AllPeriods.sgn.probes_lhvloose_EGAM1.bkg.vprobes_vlhvloose_EGAM7.GRL_v97.dataframe.h5'
dpath_data18 = '../data/output/data18_13TeV.AllPeriods.sgn.probes_lhvloose_EGAM1.bkg.vprobes_vlhvloose_EGAM7.GRL_v97.dataframe.h5'
dpath_mc16   = '../data/output/mc16_13TeV.302236_309995_341330.sgn.boosted_probes.WZ_llqq_plus_radion_ZZ_llqq_plus_ggH3000.merge.dataframe.h5'

In [4]:
tpath_data17 = 'output/emulation/data17_table_v10.h5'
tpath_data18 = 'output/emulation/data18_table_v10.h5'
tpath_mc16   = 'output/emulation/mc16_table_v10.h5'

## Setup Chains:

In [5]:
# create my chain
chains = [
            Chain( "HLT_e17_lhvloose_nod0_ringer_v10_L1EM15VHI", L1Seed = 'L1_EM15VHI' , l2calo_column = 'ringer_v10_vloose'),
            Chain( "HLT_e28_lhtight_nod0_ringer_v10_ivarloose", L1Seed = 'L1_EM22VHI' , l2calo_column = 'ringer_v10_tight' ),
            Chain( "HLT_e60_lhmedium_nod0_ringer_v10_L1EM24VHI", L1Seed = 'L1_EM24VHI' , l2calo_column = 'ringer_v10_medium'),
            Chain( "HLT_e140_lhloose_nod0_ringer_v10" , L1Seed = 'L1_EM24VHI', l2calo_column = 'ringer_v10_loose'),
]

def emulate( df ):
    for chain in progressbar(chains, prefix='Emulate...'):
        chain.apply(df)

OrderedDict([('L1Seed', 'L1_EM15VHI'),
             ('L2Calo', (14000.0, 'ringer_v10_vloose')),
             ('L2', 'trig_L2_el_cut_pt15to20'),
             ('EFCalo', 17000.0),
             ('HLT', (17000.0, 'trig_EF_el_lhvloose'))])
OrderedDict([('L1Seed', 'L1_EM22VHI'),
             ('L2Calo', (25000.0, 'ringer_v10_tight')),
             ('L2', 'trig_L2_el_cut_pt20to50'),
             ('EFCalo', 28000.0),
             ('HLT', (28000.0, 'trig_EF_el_lhtight_ivarloose'))])
OrderedDict([('L1Seed', 'L1_EM24VHI'),
             ('L2Calo', (57000.0, 'ringer_v10_medium')),
             ('L2', 'trig_L2_el_cut_pt50toInf'),
             ('EFCalo', 60000.0),
             ('HLT', (60000.0, 'trig_EF_el_lhmedium'))])
OrderedDict([('L1Seed', 'L1_EM24VHI'),
             ('L2Calo', (137000.0, 'ringer_v10_loose')),
             ('L2', 'trig_L2_el_cut_pt50toInf'),
             ('EFCalo', 140000.0),
             ('HLT', (140000.0, 'trig_EF_el_lhloose'))])


## Emulate 2017:

Take something like 1 hour

In [6]:
data17_df = pd.concat( (load_hdf(dpath_data17), load_hdf(tpath_data17)), axis=1)

In [7]:
emulate(data17_df)

Emulate...: 100%|██████████| 4/4 [01:23<00:00, 20.83s/it]


In [8]:
data17_df.shape

(43311283, 111)

### Validate:

In [9]:
eff_data17_Zee = Efficiency( output_path+'/data17_Zee_efficiency_v10.root')

2022-02-06 20:41:52,912 | Py.Efficiency                           INFO Creating the StoreGate service with path: output/emulation/data17_Zee_efficiency_v10.root


In [10]:
eff_data17_Zee.fill( data17_df.loc[data17_df['target']==1], 'HLT_e17_lhvloose_nod0_ringer_v10_L1EM15VHI', pidname='el_lhvloose', reset=True)
eff_data17_Zee.fill( data17_df.loc[data17_df['target']==1], 'HLT_e28_lhtight_nod0_ringer_v10_ivarloose' , pidname='el_lhtight' , reset=True)
eff_data17_Zee.fill( data17_df.loc[data17_df['target']==1], 'HLT_e60_lhmedium_nod0_ringer_v10_L1EM24VHI', pidname='el_lhmedium', reset=True)
eff_data17_Zee.fill( data17_df.loc[data17_df['target']==1], 'HLT_e140_lhloose_nod0_ringer_v10'          , pidname='el_lhloose' , reset=True)

Filling...: 100%|██████████| 5/5 [01:31<00:00, 18.28s/it]


HLT_e17_lhvloose_nod0_ringer_v10_L1EM15VHI
+--------+----------+-------------------+
|  Step  | Eff [%%] |    passed/total   |
+--------+----------+-------------------+
| L1Calo | 94.1069  | 30158981/32047563 |
| L2Calo | 93.5262  | 29972856/32047563 |
|   L2   | 93.2841  | 29895273/32047563 |
| EFCalo | 91.9730  | 29475103/32047563 |
|  HLT   | 90.0309  | 28852699/32047563 |
+--------+----------+-------------------+


Filling...: 100%|██████████| 5/5 [01:06<00:00, 13.22s/it]


HLT_e28_lhtight_nod0_ringer_v10_ivarloose
+--------+----------+-------------------+
|  Step  | Eff [%%] |    passed/total   |
+--------+----------+-------------------+
| L1Calo | 94.6937  | 21557594/22765602 |
| L2Calo | 94.2246  | 21450801/22765602 |
|   L2   | 94.1088  | 21424434/22765602 |
| EFCalo | 93.1837  | 21213836/22765602 |
|  HLT   | 86.9987  | 19805782/22765602 |
+--------+----------+-------------------+


Filling...: 100%|██████████| 5/5 [00:04<00:00,  1.10it/s]


HLT_e60_lhmedium_nod0_ringer_v10_L1EM24VHI
+--------+----------+-----------------+
|  Step  | Eff [%%] |   passed/total  |
+--------+----------+-----------------+
| L1Calo | 99.5079  | 1361055/1367786 |
| L2Calo | 96.7809  | 1323756/1367786 |
|   L2   | 96.6900  | 1322512/1367786 |
| EFCalo | 94.5942  | 1293846/1367786 |
|  HLT   | 91.3304  | 1249205/1367786 |
+--------+----------+-----------------+


Filling...: 100%|██████████| 5/5 [00:00<00:00, 25.30it/s]

HLT_e140_lhloose_nod0_ringer_v10
+--------+----------+--------------+
|  Step  | Eff [%%] | passed/total |
+--------+----------+--------------+
| L1Calo | 100.0000 | 60541/60541  |
| L2Calo | 95.2148  | 57644/60541  |
|   L2   | 95.1504  | 57605/60541  |
| EFCalo | 93.8752  | 56833/60541  |
|  HLT   | 92.2086  | 55824/60541  |
+--------+----------+--------------+


In [11]:
eff_data17_Zee.save()

In [12]:
eff_data17_JF17 = Efficiency( output_path+'/data17_JF17_efficiency_v10.root')

2022-02-06 20:45:22,248 | Py.Efficiency                           INFO Creating the StoreGate service with path: output/emulation/data17_JF17_efficiency_v10.root


In [13]:
eff_data17_JF17.fill( data17_df.loc[data17_df['target']==0], 'HLT_e17_lhvloose_nod0_ringer_v10_L1EM15VHI', reset=True)
eff_data17_JF17.fill( data17_df.loc[data17_df['target']==0], 'HLT_e28_lhtight_nod0_ringer_v10_ivarloose' , reset=True)
eff_data17_JF17.fill( data17_df.loc[data17_df['target']==0], 'HLT_e60_lhmedium_nod0_ringer_v10_L1EM24VHI', reset=True)
eff_data17_JF17.fill( data17_df.loc[data17_df['target']==0], 'HLT_e140_lhloose_nod0_ringer_v10'          , reset=True)

Filling...: 100%|██████████| 5/5 [00:17<00:00,  3.48s/it]


HLT_e17_lhvloose_nod0_ringer_v10_L1EM15VHI
+--------+----------+------------------+
|  Step  | Eff [%%] |   passed/total   |
+--------+----------+------------------+
| L1Calo | 84.4275  | 8738241/10349992 |
| L2Calo |  0.8554  |  88536/10349992  |
|   L2   |  0.8149  |  84341/10349992  |
| EFCalo |  0.7219  |  74721/10349992  |
|  HLT   |  0.0732  |  7575/10349992   |
+--------+----------+------------------+


Filling...: 100%|██████████| 5/5 [00:15<00:00,  3.13s/it]


HLT_e28_lhtight_nod0_ringer_v10_ivarloose
+--------+----------+-----------------+
|  Step  | Eff [%%] |   passed/total  |
+--------+----------+-----------------+
| L1Calo | 90.6060  | 8384948/9254300 |
| L2Calo |  0.2771  |  25641/9254300  |
|   L2   |  0.2542  |  23523/9254300  |
| EFCalo |  0.2119  |  19614/9254300  |
|  HLT   |  0.0008  |    75/9254300   |
+--------+----------+-----------------+


Filling...: 100%|██████████| 5/5 [00:13<00:00,  2.65s/it]


HLT_e60_lhmedium_nod0_ringer_v10_L1EM24VHI
+--------+----------+-----------------+
|  Step  | Eff [%%] |   passed/total  |
+--------+----------+-----------------+
| L1Calo | 99.9012  | 7844422/7852181 |
| L2Calo |  0.1634  |  12829/7852181  |
|   L2   |  0.1357  |  10656/7852181  |
| EFCalo |  0.0846  |   6641/7852181  |
|  HLT   |  0.0005  |    39/7852181   |
+--------+----------+-----------------+


Filling...: 100%|██████████| 5/5 [00:09<00:00,  1.94s/it]

HLT_e140_lhloose_nod0_ringer_v10
+--------+----------+-----------------+
|  Step  | Eff [%%] |   passed/total  |
+--------+----------+-----------------+
| L1Calo | 99.9227  | 5629684/5634041 |
| L2Calo |  0.3929  |  22134/5634041  |
|   L2   |  0.3405  |  19182/5634041  |
| EFCalo |  0.2276  |  12825/5634041  |
|  HLT   |  0.0102  |   577/5634041   |
+--------+----------+-----------------+


In [14]:
eff_data17_JF17.save()

In [15]:
del data17_df

## Emulate 2018:

In [16]:
data18_df = pd.concat( (load_hdf(dpath_data18), load_hdf(tpath_data18)), axis=1)

In [17]:
emulate(data18_df)

Emulate...: 100%|██████████| 4/4 [01:21<00:00, 20.35s/it]


In [18]:
data18_df.shape

(41873973, 111)

In [19]:
eff_data18_Zee = Efficiency( output_path+'/data18_Zee_efficiency_v10.root')

2022-02-06 20:50:05,321 | Py.Efficiency                           INFO Creating the StoreGate service with path: output/emulation/data18_Zee_efficiency_v10.root


In [20]:
eff_data18_Zee.fill( data18_df.loc[data18_df['target']==1], 'HLT_e17_lhvloose_nod0_ringer_v10_L1EM15VHI', pidname='el_lhvloose', reset=True)
eff_data18_Zee.fill( data18_df.loc[data18_df['target']==1], 'HLT_e28_lhtight_nod0_ringer_v10_ivarloose' , pidname='el_lhtight' , reset=True)
eff_data18_Zee.fill( data18_df.loc[data18_df['target']==1], 'HLT_e60_lhmedium_nod0_ringer_v10_L1EM24VHI', pidname='el_lhmedium', reset=True)
eff_data18_Zee.fill( data18_df.loc[data18_df['target']==1], 'HLT_e140_lhloose_nod0_ringer_v10'          , pidname='el_lhloose' , reset=True)

Filling...: 100%|██████████| 5/5 [01:30<00:00, 18.03s/it]


HLT_e17_lhvloose_nod0_ringer_v10_L1EM15VHI
+--------+----------+-------------------+
|  Step  | Eff [%%] |    passed/total   |
+--------+----------+-------------------+
| L1Calo | 94.1301  | 30705693/32620495 |
| L2Calo | 93.5700  | 30522990/32620495 |
|   L2   | 93.3309  | 30445000/32620495 |
| EFCalo | 91.9973  | 30009968/32620495 |
|  HLT   | 90.5533  | 29538919/32620495 |
+--------+----------+-------------------+


Filling...: 100%|██████████| 5/5 [01:05<00:00, 13.13s/it]


HLT_e28_lhtight_nod0_ringer_v10_ivarloose
+--------+----------+-------------------+
|  Step  | Eff [%%] |    passed/total   |
+--------+----------+-------------------+
| L1Calo | 94.6555  | 22067825/23313826 |
| L2Calo | 94.1951  | 21960480/23313826 |
|   L2   | 94.0990  | 21938088/23313826 |
| EFCalo | 93.2993  | 21751633/23313826 |
|  HLT   | 87.5752  | 20417125/23313826 |
+--------+----------+-------------------+


Filling...: 100%|██████████| 5/5 [00:04<00:00,  1.09it/s]


HLT_e60_lhmedium_nod0_ringer_v10_L1EM24VHI
+--------+----------+-----------------+
|  Step  | Eff [%%] |   passed/total  |
+--------+----------+-----------------+
| L1Calo | 99.5563  | 1375545/1381675 |
| L2Calo | 96.8297  | 1337872/1381675 |
|   L2   | 96.7395  | 1336625/1381675 |
| EFCalo | 94.6460  | 1307700/1381675 |
|  HLT   | 91.6774  | 1266684/1381675 |
+--------+----------+-----------------+


Filling...: 100%|██████████| 5/5 [00:00<00:00, 25.30it/s]


HLT_e140_lhloose_nod0_ringer_v10
+--------+----------+--------------+
|  Step  | Eff [%%] | passed/total |
+--------+----------+--------------+
| L1Calo | 99.9934  | 60385/60389  |
| L2Calo | 95.1299  | 57448/60389  |
|   L2   | 95.0554  | 57403/60389  |
| EFCalo | 93.9128  | 56713/60389  |
|  HLT   | 92.4026  | 55801/60389  |
+--------+----------+--------------+


In [21]:
eff_data18_Zee.save()

In [22]:
eff_data18_JF17 = Efficiency( output_path+'/data18_JF17_efficiency_v10.root')

2022-02-06 20:53:33,557 | Py.Efficiency                           INFO Creating the StoreGate service with path: output/emulation/data18_JF17_efficiency_v10.root


In [23]:
eff_data18_JF17.fill( data18_df.loc[data18_df['target']==0], 'HLT_e17_lhvloose_nod0_ringer_v10_L1EM15VHI', reset=True)
eff_data18_JF17.fill( data18_df.loc[data18_df['target']==0], 'HLT_e28_lhtight_nod0_ringer_v10_ivarloose' , reset=True)
eff_data18_JF17.fill( data18_df.loc[data18_df['target']==0], 'HLT_e60_lhmedium_nod0_ringer_v10_L1EM24VHI', reset=True)
eff_data18_JF17.fill( data18_df.loc[data18_df['target']==0], 'HLT_e140_lhloose_nod0_ringer_v10'          , reset=True)

Filling...: 100%|██████████| 5/5 [00:14<00:00,  2.82s/it]


HLT_e17_lhvloose_nod0_ringer_v10_L1EM15VHI
+--------+----------+-----------------+
|  Step  | Eff [%%] |   passed/total  |
+--------+----------+-----------------+
| L1Calo | 85.5183  | 7240922/8467098 |
| L2Calo |  1.1630  |  98471/8467098  |
|   L2   |  1.1243  |  95193/8467098  |
| EFCalo |  1.0317  |  87354/8467098  |
|  HLT   |  0.3913  |  33132/8467098  |
+--------+----------+-----------------+


Filling...: 100%|██████████| 5/5 [00:12<00:00,  2.55s/it]


HLT_e28_lhtight_nod0_ringer_v10_ivarloose
+--------+----------+-----------------+
|  Step  | Eff [%%] |   passed/total  |
+--------+----------+-----------------+
| L1Calo | 91.4008  | 6972786/7628804 |
| L2Calo |  0.4918  |  37517/7628804  |
|   L2   |  0.4700  |  35858/7628804  |
| EFCalo |  0.4281  |  32662/7628804  |
|  HLT   |  0.0258  |   1966/7628804  |
+--------+----------+-----------------+


Filling...: 100%|██████████| 5/5 [00:10<00:00,  2.17s/it]


HLT_e60_lhmedium_nod0_ringer_v10_L1EM24VHI
+--------+----------+-----------------+
|  Step  | Eff [%%] |   passed/total  |
+--------+----------+-----------------+
| L1Calo | 99.8995  | 6555389/6561985 |
| L2Calo |  0.3328  |  21838/6561985  |
|   L2   |  0.3062  |  20094/6561985  |
| EFCalo |  0.2574  |  16892/6561985  |
|  HLT   |  0.0275  |   1806/6561985  |
+--------+----------+-----------------+


Filling...: 100%|██████████| 5/5 [00:08<00:00,  1.62s/it]

HLT_e140_lhloose_nod0_ringer_v10
+--------+----------+-----------------+
|  Step  | Eff [%%] |   passed/total  |
+--------+----------+-----------------+
| L1Calo | 99.9159  | 4772502/4776518 |
| L2Calo |  0.6820  |  32577/4776518  |
|   L2   |  0.6306  |  30121/4776518  |
| EFCalo |  0.5207  |  24871/4776518  |
|  HLT   |  0.2004  |   9571/4776518  |
+--------+----------+-----------------+


In [24]:
eff_data18_JF17.save()

## Evaluate mc16 boosted:

In [6]:
mc16_df = pd.concat( (load_hdf(dpath_mc16), load_hdf(tpath_mc16)), axis=1)

In [7]:
emulate(mc16_df)

Emulate...: 100%|██████████| 4/4 [00:00<00:00, 36.63it/s]


In [8]:
eff_mc16 = Efficiency( output_path+'/mc16_boosted_efficiency_v10.root')

2022-02-08 20:10:33,983 | Py.Efficiency                           INFO Creating the StoreGate service with path: output/emulation/mc16_boosted_efficiency_v10.root


In [9]:
eff_mc16.fill( mc16_df.loc[mc16_df['target']==1], 'HLT_e17_lhvloose_nod0_ringer_v10_L1EM15VHI', pidname='el_lhvloose', reset=True)
eff_mc16.fill( mc16_df.loc[mc16_df['target']==1], 'HLT_e28_lhtight_nod0_ringer_v10_ivarloose' , pidname='el_lhtight' , reset=True)
eff_mc16.fill( mc16_df.loc[mc16_df['target']==1], 'HLT_e60_lhmedium_nod0_ringer_v10_L1EM24VHI', pidname='el_lhmedium', reset=True)
eff_mc16.fill( mc16_df.loc[mc16_df['target']==1], 'HLT_e140_lhloose_nod0_ringer_v10'          , pidname='el_lhloose' , reset=True)

Filling...: 100%|██████████| 5/5 [00:00<00:00, 19.24it/s]


HLT_e17_lhvloose_nod0_ringer_v10_L1EM15VHI
+--------+----------+--------------+
|  Step  | Eff [%%] | passed/total |
+--------+----------+--------------+
| L1Calo | 99.7303  | 78407/78619  |
| L2Calo | 75.7374  | 59544/78619  |
|   L2   | 75.7298  | 59538/78619  |
| EFCalo | 61.8744  | 48645/78619  |
|  HLT   | 60.9662  | 47931/78619  |
+--------+----------+--------------+


Filling...: 100%|██████████| 5/5 [00:00<00:00, 25.36it/s]


HLT_e28_lhtight_nod0_ringer_v10_ivarloose
+--------+----------+--------------+
|  Step  | Eff [%%] | passed/total |
+--------+----------+--------------+
| L1Calo | 99.8427  | 72990/73105  |
| L2Calo | 40.0438  | 29274/73105  |
|   L2   | 40.0410  | 29272/73105  |
| EFCalo | 34.7350  | 25393/73105  |
|  HLT   | 33.3739  | 24398/73105  |
+--------+----------+--------------+


Filling...: 100%|██████████| 5/5 [00:00<00:00, 24.32it/s]


HLT_e60_lhmedium_nod0_ringer_v10_L1EM24VHI
+--------+----------+--------------+
|  Step  | Eff [%%] | passed/total |
+--------+----------+--------------+
| L1Calo | 99.9635  | 73911/73938  |
| L2Calo | 45.0296  | 33294/73938  |
|   L2   | 45.0256  | 33291/73938  |
| EFCalo | 38.2158  | 28256/73938  |
|  HLT   | 37.2528  | 27544/73938  |
+--------+----------+--------------+


Filling...: 100%|██████████| 5/5 [00:00<00:00, 23.63it/s]

HLT_e140_lhloose_nod0_ringer_v10
+--------+----------+--------------+
|  Step  | Eff [%%] | passed/total |
+--------+----------+--------------+
| L1Calo | 99.9882  | 67947/67955  |
| L2Calo | 71.9167  | 48871/67955  |
|   L2   | 71.9138  | 48869/67955  |
| EFCalo | 58.3533  | 39654/67955  |
|  HLT   | 57.2835  | 38927/67955  |
+--------+----------+--------------+


In [10]:
eff_mc16.save()